In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'danilo-ai-project'

In [2]:
REGION = 'us-central1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

In [3]:
from google.cloud import bigquery
from google.cloud import storage

In [4]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

In [5]:
BUCKET = PROJECT_ID

In [6]:
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

In [7]:
bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')

The file has already been created at: gs://danilo-ai-project/01/01/data/fraud.csv


In [8]:
list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}'))

[<Blob: danilo-ai-project, 01/01/data/fraud.csv, 1765473485657105>]

In [9]:
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')

Review the files in the console here:
https://console.cloud.google.com/storage/browser/danilo-ai-project/01;tab=objects&project=danilo-ai-project


In [11]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

In [12]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

In [13]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud


In [14]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Creating Table ...
Finished creating table: danilo-ai-project.fraud.fraud


In [15]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
    #LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.00,0
1,27074,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.00,0
2,28292,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.00,0
3,28488,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.00,0
4,31392,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142396,87209,-0.024950,1.554469,1.587469,4.108145,1.293879,1.075116,1.097120,-0.314708,-2.246136,...,-0.231031,-0.369423,-0.109470,0.730357,-0.533616,-0.011651,-0.314704,-0.284703,0.78,0
142397,163484,-0.639453,1.753366,-1.503488,-0.370006,0.918963,-0.455540,0.052645,-2.587461,-0.909486,...,-0.935817,1.405463,0.023547,0.626422,-0.389439,-0.209657,0.051996,0.206556,0.78,0
142398,158709,2.124206,-0.013996,-1.597990,0.205808,0.299265,-0.975089,0.242633,-0.362167,0.644597,...,0.199132,0.748822,-0.084798,-0.617488,0.360686,-0.067693,-0.020362,-0.065791,0.78,0
142399,43587,-3.223180,3.329060,-0.089129,0.236120,-0.636745,-0.916207,0.488061,-0.033712,1.676460,...,-0.061234,0.294829,-0.022459,0.334070,0.069616,-0.452110,-0.465946,-0.100211,0.78,0


In [16]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=danilo-ai-project


In [17]:
query = f"""
    SELECT Class
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [18]:
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: Int64

In [19]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [20]:
query = f"""
    SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TEST,28272,9.926722
1,TRAIN,228107,80.091781
2,VALIDATE,28428,9.981496


In [21]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [22]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,75591,-0.895828,0.987769,2.085479,1.200862,1.130164,-0.196441,0.960378,-0.086019,-1.524370,...,-0.423554,-0.020799,0.566804,-0.072593,-0.178259,-0.171176,0.0,0,745ef57a-93d4-49a0-a977-db25f9195afd,TEST
1,143825,-0.604535,1.136361,-0.495531,-1.308734,0.823765,-0.419241,0.777352,0.419418,-0.272374,...,-0.272672,-1.120651,-0.010361,-0.066618,0.213941,0.129667,0.0,0,5c90e691-08d4-488e-80eb-f78b4db6108a,TEST
2,121835,-2.899045,-0.658546,1.557115,2.468823,2.297007,-0.079087,1.899439,-1.784427,0.937876,...,-0.781772,0.064342,-0.559740,-0.521779,-2.856085,-0.566325,0.0,0,1b30a29e-c67f-49da-b788-1d8068f7612c,TEST
3,62011,-0.674955,0.211236,1.439655,-0.345058,0.872729,-1.360790,0.812775,-0.335824,-0.082551,...,-0.250645,0.419802,0.425173,0.369290,-0.248109,-0.223121,0.0,0,95857973-89ee-4454-be1e-e3657863853a,TEST
4,12182,-0.187468,0.893039,2.443297,1.870030,0.184394,0.084015,0.315892,-0.157057,1.080801,...,0.096787,0.341439,-1.216540,-0.471877,0.074704,0.025353,0.0,0,54242b8e-086e-461f-8f81-fcd7138a738f,TEST
